In [1]:
import pandas as pd
import numpy as np
import os
import scipy
from scipy.interpolate import griddata
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.nn.functional import mse_loss
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(f"Using device: {device}")

class Swish(nn.Module):
    def __init__(self, inplace=True):
        super(Swish, self).__init__()
        self.inplace = inplace

    def forward(self, x):
        if self.inplace:
            x.mul_(torch.sigmoid(x))
            return x
        else:
            return x * torch.sigmoid(x)


class NavierStokesPINN_U(torch.nn.Module):
    def __init__(self):
        super(NavierStokesPINN_U, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(4, 50),
            Swish(),
            torch.nn.Linear(50, 50),
            Swish(),
            torch.nn.Linear(50, 50),
            Swish(),
            torch.nn.Linear(50, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

class NavierStokesPINN_V(torch.nn.Module):
    def __init__(self):
        super(NavierStokesPINN_V, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(4, 50),
            Swish(),
            torch.nn.Linear(50, 50),
            Swish(),
            torch.nn.Linear(50, 50),
            Swish(),
            torch.nn.Linear(50, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

class NavierStokesPINN_P(torch.nn.Module):
    def __init__(self):
        super(NavierStokesPINN_P, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(4, 50),
            Swish(),
            torch.nn.Linear(50, 50),
            Swish(),
            torch.nn.Linear(50, 50),
            Swish(),
            torch.nn.Linear(50, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

Using device: cuda


In [3]:
#Dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.inputs = torch.tensor(data[:, [0, 1, 5, 7]], dtype=torch.float32).to(device)
        self.targets = torch.tensor(data[:, [2, 3, 4]], dtype=torch.float32).to(device)


    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]


In [ ]:
# Define absolute paths for saved weights
weights_dir = "models"  # Use raw string (r"...") to avoid escape character issues
model_1_path = os.path.join(weights_dir, "b_329_1_epoch1_5.pth")
model_2_path = os.path.join(weights_dir, "b_329_2_epoch1_5.pth")
model_3_path = os.path.join(weights_dir, "b_329_3_epoch1_5.pth")

def initialize_models():
    # Move models to GPU
    model_1 = NavierStokesPINN_U().to(device)
    model_2 = NavierStokesPINN_V().to(device)
    model_3 = NavierStokesPINN_P().to(device)

    print(f"Model_1 is on: {next(model_1.parameters()).device}")
    print(f"Model_2 is on: {next(model_2.parameters()).device}")
    print(f"Model_3 is on: {next(model_3.parameters()).device}")


    optimizer_1 = optim.Adam(model_1.parameters(), lr=1e-3)
    optimizer_2 = optim.Adam(model_2.parameters(), lr=1e-3)
    optimizer_3 = optim.Adam(model_3.parameters(), lr=1e-3)

        # Load saved weights if they exist
    if os.path.exists(model_1_path):
        model_1.load_state_dict(torch.load(model_1_path))
        model_1.train()
        print(f"✅ Loaded weights from {model_1_path}")
    else:
        print(f"⚠️ No saved weights found at {model_1_path}. Training from scratch.")

    if os.path.exists(model_2_path):
        model_2.load_state_dict(torch.load(model_2_path))
        model_2.train()
        print(f"✅ Loaded weights from {model_2_path}")
    else:
        print(f"⚠️ No saved weights found at {model_2_path}. Training from scratch.")

    if os.path.exists(model_3_path):
        model_3.load_state_dict(torch.load(model_3_path))
        model_3.train()
        print(f"✅ Loaded weights from {model_3_path}")
    else:
        print(f"⚠️ No saved weights found at {model_3_path}. Training from scratch.")

    return model_1, model_2, model_3, optimizer_1, optimizer_2, optimizer_3

In [ ]:
def compute_distance(x, y, d, n):
    """Compute the minimum distance of (x, y) from the nearest circular post."""
    tilt = 0.4/n
    centers = [(0, 0), (0, 0.4), (0.4, tilt), (0.4, 0.4+tilt)]
    
    distances = torch.full_like(x, float("inf"))  # Initialize distances with large values

    for cx, cy in centers:
        r = d / 2
        distance_to_post = torch.sqrt((x - cx) ** 2 + (y - cy) ** 2) - r
        distances = torch.minimum(distances, distance_to_post)  # Take the minimum distance

    return torch.maximum(distances, torch.tensor(0.0, device=x.device))  # Set negative distances to zero

def criterion_pde(x_batch, model_1, model_2, model_3, mse_loss, rho=1.0, nu=0.01):
    x_batch.requires_grad_()

    u = model_1(x_batch)
    v = model_2(x_batch)
    p = model_3(x_batch)

    # First-order derivatives
    du_d = torch.autograd.grad(u, x_batch, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    dv_d = torch.autograd.grad(v, x_batch, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    dp_d = torch.autograd.grad(p, x_batch, grad_outputs=torch.ones_like(p), create_graph=True)[0]

    du_dx, du_dy = du_d[:, 0], du_d[:, 1]
    dv_dx, dv_dy = dv_d[:, 0], dv_d[:, 1]
    dp_dx, dp_dy = dp_d[:, 0], dp_d[:, 1]

    # Second-order derivatives
    du_dxx = torch.autograd.grad(du_dx, x_batch, grad_outputs=torch.ones_like(du_dx), create_graph=True)[0][:, 0]
    du_dyy = torch.autograd.grad(du_dy, x_batch, grad_outputs=torch.ones_like(du_dy), create_graph=True)[0][:, 1]
    dv_dxx = torch.autograd.grad(dv_dx, x_batch, grad_outputs=torch.ones_like(dv_dx), create_graph=True)[0][:, 0]
    dv_dyy = torch.autograd.grad(dv_dy, x_batch, grad_outputs=torch.ones_like(dv_dy), create_graph=True)[0][:, 1]

    # Navier-Stokes residuals
    # Continuity equation
    f_c = du_dx + dv_dy

    # Momentum equations
    f_u = u.squeeze() * du_dx + v.squeeze() * du_dy + (1/rho) * dp_dx - nu * (du_dxx + du_dyy)
    f_v = u.squeeze() * dv_dx + v.squeeze() * dv_dy + (1/rho) * dp_dy - nu * (dv_dxx + dv_dyy)

    # Loss for each PDE
    loss_c = mse_loss(f_c, torch.zeros_like(f_c))
    loss_u = mse_loss(f_u, torch.zeros_like(f_u))
    loss_v = mse_loss(f_v, torch.zeros_like(f_v))

    return loss_c, loss_u, loss_v

def criterion(x_batch, y_batch, model_1, model_2, model_3, mse_loss, rho=1.0, nu=0.01):
    u_pred = model_1(x_batch)
    v_pred = model_2(x_batch)
    p_pred = model_3(x_batch)

    # Data loss
    u_loss = mse_loss(u_pred, y_batch[:, 0:1])
    v_loss = mse_loss(v_pred, y_batch[:, 1:2])
    p_loss = mse_loss(p_pred, y_batch[:, 2:3])
    L_data = u_loss + v_loss + p_loss

    return L_data, u_loss, v_loss, p_loss

def criterion_p(x_batch_periodic, y_batch_periodic, model_1, model_2, model_3, mse_loss, periodic_drt, rho=1.0, nu=0.01):
    """
    Computes the loss based on physics-based PDE constraints and data loss.

    Args:
    - x_batch_p: Input tensor pairs of periodic conditions (batch of input values)
    - y_batch_p: Target tensor pairs of periodic conditions(batch of true output values)
    - model_1, model_2, model_3: Neural network models for u, v, and p
    - mse_loss: MSE loss function
    - rho: Density parameter for PDE constraints
    - nu: Viscosity parameter for PDE constraints

    Returns:
    - Total loss: Sum of physics loss and data loss
    """
    B = x_batch_periodic.shape[0]

    x_flat = x_batch_periodic.view(B*2, 4).requires_grad_()
    y_flat = y_batch_periodic   .view(B*2, 3)

    u_flat = model_1(x_flat).view(-1)
    v_flat = model_2(x_flat).view(-1)
    p_flat = model_3(x_flat).view(-1)

    x_coord = x_flat[:, 0:1]
    y_coord = x_flat[:, 1:2]
    d      = x_flat[:, 2:3]
    n      = x_flat[:, 3:4]

    distances = compute_distance(x_coord, y_coord, d, n)
    zero_mask = distances == 0

    u_flat = torch.where(zero_mask.view(-1), 0.0, u_flat)
    v_flat = torch.where(zero_mask.view(-1), 0.0, v_flat)

    v_flat = torch.where(x_coord.view(-1) == 0,   0.0, v_flat)
    p_flat = torch.where(x_coord.view(-1) == 0.4, 0.0, p_flat)

    u_pred = u_flat.view(B, 2)
    v_pred = v_flat.view(B, 2)
    p_pred = p_flat.view(B, 2)

    u_diff_loss = mse_loss(u_pred[:,0] - u_pred[:,1],
                           torch.zeros(B, device=u_pred.device))
    v_diff_loss = mse_loss(v_pred[:,0] - v_pred[:,1],
                           torch.zeros(B, device=v_pred.device))
    p_diff_loss = mse_loss(p_pred[:,0] - p_pred[:,1],
                           torch.zeros(B, device=p_pred.device))

    L_diff = periodic_drt * (u_diff_loss + v_diff_loss + p_diff_loss)

    u_data_loss = mse_loss(u_flat, y_flat[:, 0])
    v_data_loss = mse_loss(v_flat, y_flat[:, 1])
    p_data_loss = mse_loss(p_flat, y_flat[:, 2])

    L_data = u_data_loss + v_data_loss + p_data_loss

    return L_data, L_diff, u_data_loss, v_data_loss, p_data_loss, \
           u_diff_loss, v_diff_loss, p_diff_loss


In [6]:
class CustomDataset_p(Dataset):
    def __init__(self, data):
        """
        Args:
            data: array-like of shape (N, 2, 8)
        """
        # convert to float tensor if it isn't already
        self.data = torch.as_tensor(data, dtype=torch.float32)
        # specify which columns are inputs/targets
        self.input_idx  = [0, 1, 5, 7]
        self.target_idx = [2, 3, 4]

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        # sample: shape (2, 8)
        sample = self.data[idx]
        # inputs: take cols 0,1,5,7 → (2,4)
        x = sample[:, self.input_idx]
        # targets: take cols 2,3,4 → (2,3)
        y = sample[:, self.target_idx]
        return x, y

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

#loss track

def main():
    # Load dataset (modify file path accordingly)
    file_path = os.path.join(os.getcwd(), "data/merged_dataset_REU.parquet")

    # Load data using pandas and convert to NumPy array
    df = pd.read_parquet(file_path)  # Load Parquet file
    data = df.to_numpy(dtype=np.float32)  # Convert to NumPy array
    
    dataset = CustomDataset(data)
    drt = len(dataset)

    upper = 16500
    size = 18000
    num_configs = 120
    periodic_data = []

    for i in range(num_configs):
        for j in range(i*size + upper, i*size + upper + 250):
            periodic_data.append((data[j], data[j+250]))

    periodic_dataset = CustomDataset_p(np.array(periodic_data))

    periodic_drt = len(periodic_dataset)

    dataloader = DataLoader(dataset, batch_size=100000, shuffle=True)
    periodic_dataloader = DataLoader(periodic_dataset, batch_size=30000, shuffle=True)

    print(drt)
    print(periodic_drt)
    num_batches = len(dataloader) + len(periodic_dataloader)
    print(num_batches)

    # Initialize models and optimizers
    model_1, model_2, model_3, optimizer_1, optimizer_2, optimizer_3 = initialize_models()

    # Training loop
    num_epochs = 100
    mse_loss = nn.MSELoss()
    start_time = time.time()
    
    # Initialize minimum loss to a large value
    min_loss = float('inf')
    
    # Lists to store loss history
    loss_history = {
        'epoch': [],
        'total_loss': [],
        'u_loss': [],
        'v_loss': [],
        'p_loss': [],
        # periodic losses
        'periodic_total_loss': [],
        'u_diff_loss': [],
        'v_diff_loss': [],
        'p_diff_loss': [],
        'periodic_diff_total': []
    }
    
    # Loss weights
    lambda_data = 1.0
    lambda_pde = 0.1
    lambda_periodic = 1.0

    for epoch in range(num_epochs):
        total_loss, total_u_loss, total_v_loss, total_p_loss = 0, 0, 0, 0
        total_periodic_loss, sum_u_diff, sum_v_diff, sum_p_diff = 0, 0, 0, 0
        total_pde_loss = 0

        # Create iterators
        data_iter = iter(dataloader)
        periodic_iter = iter(periodic_dataloader)
        
        num_batches = len(dataloader)
        
        for _ in range(num_batches):
            # PDE loss (using a subset of data as collocation points)
            x_batch, y_batch = next(data_iter)
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            loss_c, loss_u_pde, loss_v_pde = criterion_pde(x_batch, model_1, model_2, model_3, mse_loss)
            L_pde = loss_c + loss_u_pde + loss_v_pde
            
            # Data loss
            L_data, u_loss, v_loss, p_loss = criterion(x_batch, y_batch, model_1, model_2, model_3, mse_loss)
            
            # Periodic loss
            try:
                x_batch_periodic, y_batch_periodic = next(periodic_iter)
            except StopIteration:
                periodic_iter = iter(periodic_dataloader)
                x_batch_periodic, y_batch_periodic = next(periodic_iter)
            x_batch_periodic, y_batch_periodic = x_batch_periodic.to(device), y_batch_periodic.to(device)
            L_data_p, L_diff_p, u_loss_p, v_loss_p, p_loss_p, u_diff_p, v_diff_p, p_diff_p = criterion_p(
                                x_batch_periodic, y_batch_periodic, model_1, model_2, model_3, mse_loss, periodic_drt)
            loss_periodic = L_data_p + L_diff_p

            # Total loss
            loss = lambda_data * L_data + lambda_pde * L_pde + lambda_periodic * loss_periodic
            
            optimizer_1.zero_grad()
            optimizer_2.zero_grad()
            optimizer_3.zero_grad()
            loss.backward()
            optimizer_1.step()
            optimizer_2.step()
            optimizer_3.step()
    
            # Accumulate loss values
            total_loss += loss.item()
            total_u_loss += u_loss.item()
            total_v_loss += v_loss.item()
            total_p_loss += p_loss.item()
            total_pde_loss += L_pde.item()
            total_periodic_loss += loss_periodic.item()
            sum_u_diff += u_diff_p.item()
            sum_v_diff += v_diff_p.item()
            sum_p_diff += p_diff_p.item()
    
        # Compute average losses
        avg_loss = total_loss / num_batches
        avg_u_loss = total_u_loss / num_batches
        avg_v_loss = total_v_loss / num_batches
        avg_p_loss = total_p_loss / num_batches
        avg_pde_loss = total_pde_loss / num_batches

        avg_periodic_loss = total_periodic_loss / num_batches
        avg_u_diff = sum_u_diff / len(periodic_dataloader)
        avg_v_diff = sum_v_diff / len(periodic_dataloader)
        avg_p_diff = sum_p_diff / len(periodic_dataloader)
        avg_diff_total = avg_u_diff + avg_v_diff + avg_p_diff
        
        # Store losses in history
        loss_history['epoch'].append(epoch)
        loss_history['total_loss'].append(avg_loss)
        loss_history['u_loss'].append(avg_u_loss)
        loss_history['v_loss'].append(avg_v_loss)
        loss_history['p_loss'].append(avg_p_loss)
        loss_history['periodic_total_loss'].append(avg_periodic_loss)
        loss_history['u_diff_loss'].append(avg_u_diff)
        loss_history['v_diff_loss'].append(avg_v_diff)
        loss_history['p_diff_loss'].append(avg_p_diff)
        loss_history['periodic_diff_total'].append(avg_diff_total)
        
        # Save models if the current loss is lower than the minimum loss
        if avg_loss < min_loss:
            min_loss = avg_loss
            torch.save(model_1.state_dict(), f'models/b_329_1_epoch1_{epoch}.pth')
            torch.save(model_2.state_dict(), f'models/b_329_2_epoch1_{epoch}.pth')
            torch.save(model_3.state_dict(), f'models/b_329_3_epoch1_{epoch}.pth')
            print(f"Epoch {epoch}: New minimum loss {min_loss:.7f}, models saved")
    
        if epoch % 2 == 0:
            elapsed_time = time.time() - start_time
            print(f"Epoch [{epoch}/{num_epochs}], Avg Loss: {avg_loss:.7f}, Avg PDE Loss: {avg_pde_loss:.7f}, "
                  f"Avg Periodic: {avg_periodic_loss:.7f}, Time: {elapsed_time:.2f} sec")
            start_time = time.time()

    # Save losses to CSV
    loss_df = pd.DataFrame(loss_history)
    loss_df.to_csv('training_losses_wang.csv', index=False)
    
    # Plot loss curves
    plt.figure(figsize=(12, 8))
    plt.plot(loss_history['epoch'], loss_history['total_loss'], label='Total Loss')
    plt.plot(loss_history['epoch'], loss_history['u_loss'], label='U Loss')
    plt.plot(loss_history['epoch'], loss_history['v_loss'], label='V Loss')
    plt.plot(loss_history['epoch'], loss_history['p_loss'], label='P Loss')
    plt.plot(loss_history['epoch'], loss_history['periodic_total_loss'], label='Periodic Total Loss')
    plt.plot(loss_history['epoch'], loss_history['periodic_diff_total'], label='Periodic Diff Total')
    plt.plot(loss_history['epoch'], loss_history['u_diff_loss'], label='U Diff Loss')
    plt.plot(loss_history['epoch'], loss_history['v_diff_loss'], label='V Diff Loss')
    plt.plot(loss_history['epoch'], loss_history['p_diff_loss'], label='P Diff Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss vs Epoch')
    plt.legend()
    plt.yscale('log')  # Use log scale for better visualization
    plt.grid(True)
    plt.savefig('loss_curves_wang.png')
    plt.show()

    return model_1, model_2, model_3

In [ ]:
model_1, model_2, model_3 = main()

In [ ]:
# Set models to evaluation mode
model_1.eval()
model_2.eval()
model_3.eval()

In [ ]:
# PREDICTION USING HARD ENFORCEMENT

# File path
file_path = "data/csv/DRPINN_0.3_3_1.csv"

# Load CSV file
df = pd.read_csv(file_path)

# Ensure the correct columns are selected for input
input_data = df.iloc[:, [0, 1, 5, 7]].values  # Selecting columns (x, y, d, N)
output_data = df.iloc[:, [2, 3, 4]].values  # Selecting columns (u, v, p)

# Convert to PyTorch tensor and move to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)

# Define function to compute the minimum distance from the nearest circular post
def compute_distance(x, y, d, n):
    """Compute the minimum distance of (x, y) from the nearest circular post."""
    tilt = 0.4 / n
    centers = [(0, 0), (0, 0.4), (0.4, tilt), (0.4, 0.4 + tilt)]
    
    distances = torch.full_like(x, float("inf"))  # Initialize distances with large values

    for cx, cy in centers:
        r = d / 2
        distance_to_post = torch.sqrt((x - cx) ** 2 + (y - cy) ** 2) - r
        distances = torch.minimum(distances, distance_to_post)  # Take the minimum distance

    return torch.maximum(distances, torch.tensor(0.0, device=device))  # Set negative distances to zero

# Make predictions
with torch.no_grad():
    u_pred = model_1(input_tensor)
    v_pred = model_2(input_tensor)
    p_pred = model_3(input_tensor)

# Extract individual elements
x, y, d, n = input_tensor[:, 0], input_tensor[:, 1], input_tensor[:, 2], input_tensor[:, 3]

# Apply conditions
v_pred[x == 0] = 0  # If x = 0, set v_pred to zero

# Compute distance
distances = compute_distance(x, y, d, n)

# If distance is zero, set u and v to zero
mask = distances == 0
u_pred[mask] = 0
v_pred[mask] = 0

# Move predictions to CPU and convert to NumPy
u_pred = u_pred.cpu().numpy()
v_pred = v_pred.cpu().numpy()
p_pred = p_pred.cpu().numpy()


In [ ]:
#prediction for NO MODIFICATION
import pandas as pd
import torch
import numpy as np

# File path
file_path = "data/csv/DRPINN_0.3_3_1.csv"

# Load CSV file
df = pd.read_csv(file_path)

# Ensure the correct columns are selected for input (same format as training)
input_data = df.iloc[:, [0, 1, 5, 7]].values  # Selecting columns (x, y, d)
output_data = df.iloc[:, [2, 3, 4]].values  # Selecting columns (x, y, d)

# Convert to PyTorch tensor and move to GPU (if available)
input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)

with torch.no_grad():
    u_pred = model_1(input_tensor)
    v_pred = model_2(input_tensor)
    p_pred = model_3(input_tensor)

# Move predictions to CPU and convert to NumPy
u_pred = u_pred.cpu().numpy()
v_pred = v_pred.cpu().numpy()
p_pred = p_pred.cpu().numpy()


In [ ]:
x_grid = input_data[:, 0].reshape(-1)
y_grid = input_data[:, 1].reshape(-1)
v_actual_grid = output_data[:, 1].reshape(-1)
v_pred_grid = v_pred.flatten()
u_actual_grid = output_data[:, 0].reshape(-1)
u_pred_grid = u_pred.flatten()

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    'x': x_grid,
    'y': y_grid,
    'u': u_pred_grid,
    'v': v_pred_grid
})

# Save to CSV
df.to_csv('data/csv/vW_0.55_14.csv', index=False)

print("CSV file saved successfully!")

In [ ]:
v_pred_grid.shape

In [ ]:
%matplotlib inline

# Determine the common range for colorbar
vmin = min(v_actual_grid.min(), v_pred_grid.min())
vmax = max(v_actual_grid.max(), v_pred_grid.max())

# Plotting
plt.figure(figsize=(12, 6))

# Actual u-velocity
plt.subplot(1, 2, 1)
plt.tricontourf(x_grid, y_grid, v_actual_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Actual u-velocity')
plt.xlabel('x')
plt.ylabel('y')

# Predicted u-velocity
plt.subplot(1, 2, 2)
plt.tricontourf(x_grid, y_grid, v_pred_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Predicted u-velocity')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.show()


In [ ]:
%matplotlib inline

# Determine the common range for colorbar
vmin = min(v_actual_grid.min(), v_pred_grid.min())
vmax = max(v_actual_grid.max(), v_pred_grid.max())

# Plotting
plt.figure(figsize=(12, 6))

# Actual u-velocity
plt.subplot(1, 2, 1)
plt.tricontourf(x_grid, y_grid, v_actual_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Actual u-velocity')
plt.xlabel('x')
plt.ylabel('y')

# Predicted u-velocity
plt.subplot(1, 2, 2)
plt.tricontourf(x_grid, y_grid, v_pred_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Predicted u-velocity')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.show()


In [ ]:
%matplotlib inline

# Determine the common range for colorbar
vmin = min(v_actual_grid.min(), v_pred_grid.min())
vmax = max(v_actual_grid.max(), v_pred_grid.max())

# Plotting
plt.figure(figsize=(12, 6))

# Actual u-velocity
plt.subplot(1, 2, 1)
plt.tricontourf(x_grid, y_grid, v_actual_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Actual u-velocity')
plt.xlabel('x')
plt.ylabel('y')

# Predicted u-velocity
plt.subplot(1, 2, 2)
plt.tricontourf(x_grid, y_grid, v_pred_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Predicted u-velocity')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.show()


In [ ]:
torch.save(model_1.state_dict(), "models/model_1_weights_3i30i1.pth")
torch.save(model_2.state_dict(), "models/model_2_weights_3i30i1.pth")
torch.save(model_3.state_dict(), "models/model_3_weights_3i30i1.pth")

In [ ]:
#actual

def main():
    # Load dataset (modify file path accordingly)
    file_path = os.path.join(os.getcwd(), "data/merged_dataset_REU.parquet")

    # Load data using pandas and convert to NumPy array
    df = pd.read_parquet(file_path)  # Load Parquet file
    data = df.to_numpy(dtype=np.float32)  # Convert to NumPy array
    
    dataset = CustomDataset(data)
    drt = len(dataset)
    dataloader = DataLoader(dataset, batch_size=10000, shuffle=True)
    print(drt)
    num_batches = len(dataloader)
    print(num_batches)

    # Initialize models and optimizers
    model_1, model_2, model_3, optimizer_1, optimizer_2, optimizer_3 = initialize_models()

    # Training loop
    num_epochs = 250
    mse_loss = nn.MSELoss()
    start_time = time.time()
    
    # Initialize minimum loss to a large value
    min_loss = float('inf')
    
    for epoch in range(num_epochs):
        total_loss, total_u_loss, total_v_loss, total_p_loss = 0, 0, 0, 0
        num_batches = len(dataloader)
        
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            loss, u_loss, v_loss, p_loss = criterion(x_batch, y_batch, model_1, model_2, model_3, mse_loss)
    
            optimizer_1.zero_grad()
            optimizer_2.zero_grad()
            optimizer_3.zero_grad()
            loss.backward()
            optimizer_1.step()
            optimizer_2.step()
            optimizer_3.step()
    
            # Accumulate loss values
            total_loss += loss.item()
            total_u_loss += u_loss.item()
            total_v_loss += v_loss.item()
            total_p_loss += p_loss.item()
    
        # Compute average loss
        avg_loss = total_loss / num_batches
        avg_u_loss = total_u_loss / num_batches
        avg_v_loss = total_v_loss / num_batches
        avg_p_loss = total_p_loss / num_batches
        
        # Save models if the current loss is lower than the minimum loss
        if avg_loss < min_loss:
            min_loss = avg_loss
            torch.save(model_1.state_dict(), f'best_model_1_epoch3_{epoch}.pth')
            torch.save(model_2.state_dict(), f'best_model_2_epoch3_{epoch}.pth')
            torch.save(model_3.state_dict(), f'best_model_3_epoch3_{epoch}.pth')
            print(f"Epoch {epoch}: New minimum loss {min_loss:.7f}, models saved")
    
        if epoch % 2 == 0:
            elapsed_time = time.time() - start_time
            print(f"Epoch [{epoch}/{num_epochs}], Avg Loss: {avg_loss:.7f}, Avg ULoss: {avg_u_loss:.7f}, "
                  f"Avg VLoss: {avg_v_loss:.7f}, Avg PLoss: {avg_p_loss:.7f}, Time: {elapsed_time:.2f} sec")
            start_time = time.time()

    return model_1, model_2, model_3
